# Skin Disease Analyzer on MCNN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [ ]:
img_width, img_height = 200, 200

In [ ]:
train_data_dir = "drive/MyDrive/train"
validation_data_dir = "drive/MyDrive/test"
nb_train_samples = 1600
nb_validation_samples = 400
epochs = 112
batch_size = 30 #16

In [ ]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1. / 255,
    shear_range = 0.2,
    zoom_range= 0.2,
    horizontal_flip= True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size= (img_width, img_height),
    batch_size= batch_size,
    class_mode= 'categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size= (img_width, img_height),
    batch_size= batch_size,
    class_mode= 'categorical')

Found 1600 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 198, 198, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 97, 97, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        1

In [ ]:
model.compile(loss= 'categorical_crossentropy',
    optimizer= 'adam',
    metrics= ['accuracy'] )

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch= nb_train_samples // batch_size,
    epochs= epochs,
    validation_data= validation_generator,
    validation_steps= nb_validation_samples // batch_size)
model.save_weights("drive/MyDrive/SDA.h5")
print("Completed!!")

Epoch 1/112
53/53 [==============================] - 529s 9s/step - loss: 1.3930 - accuracy: 0.2563 - val_loss: 1.3849 - val_accuracy: 0.2487
Epoch 2/112
53/53 [==============================] - 24s 462ms/step - loss: 1.3677 - accuracy: 0.2737 - val_loss: 1.1603 - val_accuracy: 0.4821
Epoch 3/112
53/53 [==============================] - 24s 461ms/step - loss: 1.1402 - accuracy: 0.4870 - val_loss: 1.1156 - val_accuracy: 0.4641
Epoch 4/112
53/53 [==============================] - 24s 457ms/step - loss: 1.0256 - accuracy: 0.5489 - val_loss: 0.9746 - val_accuracy: 0.5385
Epoch 5/112
53/53 [==============================] - 24s 458ms/step - loss: 1.0543 - accuracy: 0.5525 - val_loss: 1.0757 - val_accuracy: 0.4513
Epoch 6/112
53/53 [==============================] - 24s 458ms/step - loss: 1.0139 - accuracy: 0.5434 - val_loss: 0.9680 - val_accuracy: 0.5538
Epoch 7/112
53/53 [==============================] - 24s 460ms/step - loss: 0.9259 - accuracy: 0.5977 - val_loss: 0.9342 - val_accuracy: 0

In [ ]:
Y_pred = model.predict(validation_generator, 
                       nb_validation_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['Acne and Rosacea', 'Eczema', 
                'Melanoma Skin Cancer, Nevi and Moles', 
                'Psoriasis pictures Lichen Planus and related diseases']
print(classification_report(validation_generator.classes, y_pred, 
                            target_names=target_names))

Confusion Matrix
[[36 19 23 22]
 [31 27 24 18]
 [24 30 22 24]
 [30 38 16 16]]
Classification Report
                                                       precision    recall  f1-score   support

                                     Acne and Rosacea       0.30      0.36      0.33       100
                                               Eczema       0.24      0.27      0.25       100
                 Melanoma Skin Cancer, Nevi and Moles       0.26      0.22      0.24       100
Psoriasis pictures Lichen Planus and related diseases       0.20      0.16      0.18       100

                                             accuracy                           0.25       400
                                            macro avg       0.25      0.25      0.25       400
                                         weighted avg       0.25      0.25      0.25       400

